In [1]:
import numpy as np
import pandas as pd

In [2]:
ratings = pd.read_csv('ratings.csv')

In [3]:
print(ratings)

      Unnamed: 0            맛집              코드         닉네임  Unnamed: 0.1   평점  \
0              0          쭈불쭈불  Num=a_nam_0138      Abbbbb        3966.0  5.0   
1              1          쭈불쭈불  Num=a_nam_0138        ^__^        3963.0  5.0   
2              2          쭈불쭈불  Num=a_nam_0138    kookkook        3957.0  3.0   
3              3          쭈불쭈불  Num=a_nam_0138       manbo        3968.0  4.0   
4              4          쭈불쭈불  Num=a_nam_0138        검은여울        3961.0  4.0   
...          ...           ...             ...         ...           ...  ...   
6897        6897           제기집   Num=a_nam_927  lemondrop1        7196.0  5.0   
6898        6898           제기집   Num=a_nam_927        직무유기        7197.0  5.0   
6899        6899          테리키친   Num=a_nam_931        할거없넹        7557.0  5.0   
6900        6900           홍릉각  Num=a_nam_1119         부처님        7350.0  5.0   
6901        6901  황박사 통마늘오리주물럭  Num=a_nam_1109         씨제이        7580.0  5.0   

      user_id  user_emb_id 

In [4]:
ratings.drop(['Unnamed: 0'], axis='columns', inplace=True)

In [5]:
print(ratings)

                맛집              코드         닉네임  Unnamed: 0.1   평점  user_id  \
0             쭈불쭈불  Num=a_nam_0138      Abbbbb        3966.0  5.0        1   
1             쭈불쭈불  Num=a_nam_0138        ^__^        3963.0  5.0        2   
2             쭈불쭈불  Num=a_nam_0138    kookkook        3957.0  3.0        3   
3             쭈불쭈불  Num=a_nam_0138       manbo        3968.0  4.0        4   
4             쭈불쭈불  Num=a_nam_0138        검은여울        3961.0  4.0        5   
...            ...             ...         ...           ...  ...      ...   
6897           제기집   Num=a_nam_927  lemondrop1        7196.0  5.0     2253   
6898           제기집   Num=a_nam_927        직무유기        7197.0  5.0     2254   
6899          테리키친   Num=a_nam_931        할거없넹        7557.0  5.0     2409   
6900           홍릉각  Num=a_nam_1119         부처님        7350.0  5.0     2497   
6901  황박사 통마늘오리주물럭  Num=a_nam_1109         씨제이        7580.0  5.0     2499   

      user_emb_id  place_id  place_emb_id  
0               0  

In [6]:
matrix = ratings.pivot_table(index='user_emb_id', columns='코드', values='평점')

In [7]:
print(matrix)

코드           Num=a_nam_0001  Num=a_nam_0002  Num=a_nam_0003  Num=a_nam_0005  \
user_emb_id                                                                   
0                       NaN             NaN             NaN             NaN   
1                       NaN             5.0             NaN             5.0   
2                       NaN             3.0             NaN             NaN   
3                       NaN             4.0             NaN             NaN   
4                       NaN             4.0             NaN             4.0   
...                     ...             ...             ...             ...   
2513                    NaN             NaN             NaN             NaN   
2514                    NaN             NaN             NaN             NaN   
2515                    NaN             NaN             NaN             NaN   
2516                    NaN             NaN             NaN             NaN   
2517                    NaN             NaN         

In [8]:
shuffled_ratings = ratings.sample(frac=1.)

In [9]:
print(shuffled_ratings)

                         맛집              코드            닉네임  Unnamed: 0.1   평점  \
5159              감성밥집 도란도란   Num=a_nam_745        숑알숑알싸리잎        6681.0  3.0   
6138                    투나스  Num=a_nam_0058        DrinkMe        6456.0  3.0   
2729                  곰이네국밥   Num=a_nam_735         케익먹고싶다        1640.0  5.0   
6234               향나무집 고대점  Num=a_nam_0077         I9C3G6        7405.0  4.0   
2185                  고래돈까스  Num=a_nam_0247             소포        3387.0  4.0   
...                     ...             ...            ...           ...  ...   
263   ¸¾½ºÅÍÄ¡ Å¸ÀÌ°ÅÇÃ¶óÀÚ  Num=a_nam_0216        ewldndm        6266.0  5.0   
5743                  핸썸베이글   Num=a_nam_727            컴포트        5427.0  4.0   
6638                  로라방앗간   Num=a_nam_881          푹자고싶다        6837.0  4.0   
6170                 마루가메제면  Num=a_nam_0360           돼지근육        5630.0  5.0   
3587                    주유소  Num=a_nam_0157  brainstorming        1932.0  4.0   

      user_id  user_emb_id 

In [10]:
USERS = shuffled_ratings['user_emb_id'].values
PLACES = shuffled_ratings['place_emb_id'].values
#[USERS, PLACES] : input(X)
RATINGS = shuffled_ratings['평점'].values
# Y
print(USERS, USERS.shape)
print(PLACES, PLACES.shape)
print(RATINGS, RATINGS.shape)

[ 143   85 1068 ... 1439  167  524] (6902,)
[164 245  40 ... 302 249  76] (6902,)
[3. 3. 5. ... 4. 5. 4.] (6902,)


In [11]:
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.layers import Embedding, Reshape, Merge
from keras.models import Sequential



Using TensorFlow backend.


In [12]:

# user latent matrix와 item latent matrix를 각자의 embedding layer을 통해 학습한 후, merge layer에서 dot product 한다. 
class DLmodel(Sequential):

    def __init__(self, n_users, m_items, k_factors, **kwargs):
        P = Sequential()
        P.add(Embedding(n_users, k_factors, input_length=1))
        P.add(Reshape((k_factors,)))

        Q = Sequential()
        Q.add(Embedding(m_items, k_factors, input_length=1))
        Q.add(Reshape((k_factors,)))

        super(DLmodel, self).__init__(**kwargs)
        self.add(Merge([P, Q], mode='dot', dot_axes=1))

    def rate(self, user_id, item_id):
        return self.predict([np.array([user_id]), np.array([item_id])])[0][0]

In [13]:
print(matrix.shape)

(2518, 410)


In [14]:
max_userid = ratings['user_id'].drop_duplicates().max()
max_placeid = ratings['place_id'].drop_duplicates().max()


In [15]:
print(max_userid, max_placeid)

2518 410


In [16]:
# model = CFModel(max_userid, max_placeid, 100)
# model.compile(loss='mse', optimizer='adamax')
#kfold 시에 주석처리 해야함(model을 for문 내에서 여러번 build함=>best model선정)
#model = CFModel(max_userid, max_placeid, 100)
#model.compile(loss='mse', optimizer='adamax')

In [17]:
callbacks = [EarlyStopping('val_loss', patience=2)]

In [20]:
# k-fold cross-validation
#train 과 test를 나눈다.
#이후 train에서 validation split까지 해준후, overfitt 방지를 위해 callback을 걸어둠.
from sklearn.model_selection import KFold
from keras.models import load_model
n_split = 10
best_model = None
error = None
for train_index,test_index in KFold(n_split).split(users):
    users_train,users_test= users[train_index], users[test_index]
    places_train,places_test = places[train_index], places[test_index]
    ratings_train,ratings_test = ratings[train_index],ratings[test_index]
    
    model = DLmodel(2518, 410, 10)
    model.compile(loss='mse', optimizer='adamax')
    model.fit([users_train, places_train], ratings_train, nb_epoch=400, 
              validation_data = ([users_test, places_test], ratings_test), 
              callbacks = callbacks)
    model_error = model.evaluate([users_test, places_test], ratings_test)
    
# history = model.fit([USERS, PLACES], RATINGS, nb_epoch=100, validation_split=.1, verbose=2, callbacks=callbacks)
#model은 array 두개를 받아야 함(USERS, RATINGS)
#데이터셋이 적어서 전체 데이터셋을 학습하고 싶은데, validation set을 나누지 않으면, overfit방지 earlystop을 할수 없으므로, 
#overfit직전 까지 학습하기 위한 최적의 epoch수를 찾기위해 kfold 통해 찾기(평균)  

Train on 6211 samples, validate on 691 samples
Epoch 1/400
6211/6211 [==============================] - 0s - loss: 18.0625 - val_loss: 18.2511
Epoch 2/400
6211/6211 [==============================] - 0s - loss: 18.0554 - val_loss: 18.2501
Epoch 3/400
6211/6211 [==============================] - 0s - loss: 18.0455 - val_loss: 18.2467
Epoch 4/400
6211/6211 [==============================] - 0s - loss: 18.0288 - val_loss: 18.2390
Epoch 5/400
6211/6211 [==============================] - 0s - loss: 18.0020 - val_loss: 18.2245
Epoch 6/400
6211/6211 [==============================] - 0s - loss: 17.9628 - val_loss: 18.2006
Epoch 7/400
6211/6211 [==============================] - 0s - loss: 17.9086 - val_loss: 18.1654
Epoch 8/400
6211/6211 [==============================] - 0s - loss: 17.8361 - val_loss: 18.1180
Epoch 9/400
6211/6211 [==============================] - 0s - loss: 17.7453 - val_loss: 18.0579
Epoch 10/400
6211/6211 [==============================] - 0s - loss: 17.6358 - val_loss: 

6211/6211 [==============================] - 0s - loss: 0.6684 - val_loss: 7.1720
Epoch 171/400
6211/6211 [==============================] - 0s - loss: 0.6595 - val_loss: 7.1680
Epoch 172/400
6211/6211 [==============================] - 0s - loss: 0.6507 - val_loss: 7.1645
Epoch 173/400
6211/6211 [==============================] - 0s - loss: 0.6419 - val_loss: 7.1609
Epoch 174/400
6211/6211 [==============================] - 0s - loss: 0.6335 - val_loss: 7.1557
Epoch 175/400
6211/6211 [==============================] - 0s - loss: 0.6252 - val_loss: 7.1548
Epoch 176/400
6211/6211 [==============================] - 0s - loss: 0.6169 - val_loss: 7.1483
Epoch 177/400
6211/6211 [==============================] - 0s - loss: 0.6091 - val_loss: 7.1476
Epoch 178/400
6211/6211 [==============================] - 0s - loss: 0.6013 - val_loss: 7.1418
Epoch 179/400
6211/6211 [==============================] - 0s - loss: 0.5936 - val_loss: 7.1415
Epoch 180/400
6211/6211 [=============================

6211/6211 [==============================] - 0s - loss: 1.9549 - val_loss: 7.8032
Epoch 107/400
6211/6211 [==============================] - 0s - loss: 1.9174 - val_loss: 7.7826
Epoch 108/400
6211/6211 [==============================] - 0s - loss: 1.8806 - val_loss: 7.7635
Epoch 109/400
6211/6211 [==============================] - 0s - loss: 1.8446 - val_loss: 7.7440
Epoch 110/400
6211/6211 [==============================] - 0s - loss: 1.8094 - val_loss: 7.7253
Epoch 111/400
6211/6211 [==============================] - 0s - loss: 1.7752 - val_loss: 7.7073
Epoch 112/400
6211/6211 [==============================] - 0s - loss: 1.7416 - val_loss: 7.6912
Epoch 113/400
6211/6211 [==============================] - 0s - loss: 1.7089 - val_loss: 7.6739
Epoch 114/400
6211/6211 [==============================] - 0s - loss: 1.6769 - val_loss: 7.6576
Epoch 115/400
6211/6211 [==============================] - 0s - loss: 1.6456 - val_loss: 7.6436
Epoch 116/400
6211/6211 [=============================

KeyboardInterrupt: 

In [21]:
dlmodel = DLmodel(2518, 410,10)
dlmodel.compile(loss='mse', optimizer='adamax')
dlmodel.fit([users, places], ratings, nb_epoch=220)
# 전체 데이터셋을 학습

Epoch 1/220
6902/6902 [==============================] - 0s - loss: 18.0809     
Epoch 2/220
6902/6902 [==============================] - 0s - loss: 18.0743     
Epoch 3/220
6902/6902 [==============================] - 0s - loss: 18.0649     
Epoch 4/220
6902/6902 [==============================] - 0s - loss: 18.0480     
Epoch 5/220
6902/6902 [==============================] - 0s - loss: 18.0185     
Epoch 6/220
6902/6902 [==============================] - 0s - loss: 17.9721     
Epoch 7/220
6902/6902 [==============================] - 0s - loss: 17.9062     
Epoch 8/220
6902/6902 [==============================] - 0s - loss: 17.8177     
Epoch 9/220
6902/6902 [==============================] - 0s - loss: 17.7043     
Epoch 10/220
6902/6902 [==============================] - 0s - loss: 17.5659     
Epoch 11/220
6902/6902 [==============================] - 0s - loss: 17.4009     
Epoch 12/220
6902/6902 [==============================] - 0s - loss: 17.2073     
Epoch 13/220
6902/6902 [=

6902/6902 [==============================] - 0s - loss: 0.4368     
Epoch 202/220
6902/6902 [==============================] - 0s - loss: 0.4327     
Epoch 203/220
6902/6902 [==============================] - 0s - loss: 0.4287     
Epoch 204/220
6902/6902 [==============================] - 0s - loss: 0.4248     
Epoch 205/220
6902/6902 [==============================] - 0s - loss: 0.4209     
Epoch 206/220
6902/6902 [==============================] - 0s - loss: 0.4172     
Epoch 207/220
6902/6902 [==============================] - 0s - loss: 0.4135     
Epoch 208/220
6902/6902 [==============================] - 0s - loss: 0.4099     
Epoch 209/220
6902/6902 [==============================] - 0s - loss: 0.4061     
Epoch 210/220
6902/6902 [==============================] - 0s - loss: 0.4027     
Epoch 211/220
6902/6902 [==============================] - 0s - loss: 0.3993     
Epoch 212/220
6902/6902 [==============================] - 0s - loss: 0.3958     
Epoch 213/220
6902/6902 [=====

In [22]:
def predict_rating(user_id, place_id):
    return MODEL.rate(user_id - 1, place_id - 1)

In [28]:
random_user = ratings[ratings['user_id'] == 5][['맛집','코드', 'place_id', '닉네임','user_id', '평점']]
random_user

,맛집,코드,place_id,닉네임,user_id,평점
4,쭈불쭈불,Num=a_nam_0138,1,검은여울,5,4.0
27,애기능생활관 학식,Num=a_nam_935,2,검은여울,5,5.0
150,우승식당,Num=a_nam_787,4,검은여울,5,4.0
269,고대맛집ㅋㅋ,Num=a_nam_0211,8,검은여울,5,5.0
335,고른햇살,Num=a_nam_0002,9,검은여울,5,4.0
...,...,...,...,...,...,...
3803,트라타,Num=a_nam_740,82,검은여울,5,4.0
3837,프로마치,Num=a_nam_0040,83,검은여울,5,3.0
3894,학생회관 2층,Num=a_nam_766,84,검은여울,5,4.0
3910,학생회관 자율식당,Num=a_nam_916,85,검은여울,5,5.0


In [30]:
random_user['예상평점'] = random_user.apply(lambda x: predict_rating(5, x['place_id']), axis=1)
random_user.sort_values(by='평점', ascending=False).head(60)


,맛집,코드,place_id,닉네임,user_id,평점,예상평점
2317,부부 바지락 손칼국수,Num=a_nam_0314,32,검은여울,5,5.0,3.990009
3352,이공김밥,Num=a_nam_605,71,검은여울,5,5.0,4.537937
3023,밀플랜비 (Meal Plan B),Num=a_nam_948,55,검은여울,5,5.0,4.648077
3151,서브웨이 고려대점,Num=a_nam_0045,60,검은여울,5,5.0,3.437020
2784,더멜팅,Num=a_nam_801,46,검은여울,5,5.0,4.675076
2762,나정순할매쭈꾸미,Num=a_nam_845,44,검은여울,5,5.0,5.105757
3202,안동반점,Num=a_nam_0370,64,검은여울,5,5.0,4.459789
3223,야마토텐동,Num=a_nam_817,67,검은여울,5,5.0,3.633264
2564,고고인디안쿠진 1호점,Num=a_nam_711,38,검은여울,5,5.0,4.832880
3328,오월키친,Num=a_nam_996,69,검은여울,5,5.0,4.827754


In [31]:
recommendations = ratings[ratings['place_id'].isin(random_user['place_id']) == False][['place_id', '맛집']].drop_duplicates()
recommendations['예상평점'] = recommendations.apply(lambda x: predict_rating(8, x['place_id']), axis=1)
recommendations.sort_values(by='예상평점', ascending=False).head(20)


,place_id,맛집,예상평점
5469,183,남이네 분식,5.518865
4215,106,원진노기순 청국장,5.488256
6088,240,동네,5.344956
5010,154,특별식당,5.325227
5406,178,교우회관 학생식당,5.277291
4787,138,야순네 식당,5.245344
5320,173,우리동네만두가게,5.183782
4941,151,참새방앗간,5.172176
5661,199,프릭타이,5.113809
4906,148,제기돈,5.056477
